# IBM Cloud Hyper Protect Virtual Servers Secure Build lab (with Docker Hub)

This lab will introduce you to securely building applications on the IBM Cloud Hyper Protect Virtual Servers for VPC service.

The _Secure Build_ feature lets you build, and digitally sign, a Docker image within an HPVS instance running an IBM-provided _secure build server_ that offers the confidentiality, integrity and auditability protections inherent in IBM Cloud Hyper Protect Virtual Servers for VPC.  

In this lab you will:

1. Deploy the secure build server in your own HPVS instance.
2. You will use this secure build server to securely create and digitally sign a Docker image that contains a sample application. For this lab you will use a sample _secure bitcoin wallet application_ provided by IBM for demonstration purposes.
3. You will deploy this Docker image that you've built in a second HPVS instance and then run the sample secure bitcoin wallet application contained within the image. You will configure your HPVS instance to use an **_IBM Cloud Hyper Protect Crypto Services_** (HPCS) instance for key generation.  The HPCS instance provides the security of a _FIPS 140-2 level 4_-certified _Hardware Security Module_, which is the strongest level of FIPS 140-2 certification commercially available.

You will be using an open-source application called _Jupyter Notebook_ to run this lab. Jupyter Notebook allows you to enter commands and see the results of these commands. You are likely reading this within Jupyter Notebook right now.  You will also open a terminal within Jupyter and enter some commands from the terminal. The large majority of the commands will be run from within the Jupyter notebook while a small number of commands will be run from the terminal. Those commands that prompt for user input are run from the terminal, while the vast majority, which don't prompt for user input, are run from within Jupyter Notebook.

## Secure Build overview

The diagram below provides an overview of the major components involved in the secure build process that you will be working with in this lab. An explanation of each numbered annotation is provided below the diagram.

![Secure Build architecture](images/sbsarch.png)

1. The IBM Cloud command line interface (CLI) will be used throughout this lab.  It has already been installed for you in the lab environment.

2. You will be using your IBM Cloud account during the lab.

3. The _secure build_ CLI is a separate CLI used exclusively for the Secure Build process.  You will download this CLI in _Step 5.1_ of this lab and will use it throughout the remainder of the lab.

4. You will be building a container image from an application whose source code is in a GitHub repository. The GitHub repository is specified for you in the configuration file you will build in _Step 5.5_ of this lab.  It is the value of the _GITHUB\_URL_ key given for you in _Step 5.5_. 

5. The container image that you build will be pushed to the _Docker Hub_.

6. The secure build server runs inside a Hyper Protect Virtual Servers instance in IBM Cloud that you will create in _Step 5.11_ of the lab.

7. Your secure build server contains a snapshot of its state.  You can download this _secure build state_ from the server and use it to rebuild the secure build server if necessary.  Although we won't be using this downloaded state in the lab, you will perform the command to download it in _Step 6.10_ of the lab.

**Note:** It is also possible to save the _secure build state_ to _IBM Cloud Object Storage_, but we will not be doing that in this lab.

Okay, let's get started!!

## Initial login to IBM Cloud via the _ibmcloud_ command line interface (CLI)

### Step 1.1 Open a terminal with the Jupyter Launcher

In the browser tab in which you are accessing the Jupyter Notebook to take this lab, you should see a workspace with two tabs:

    1. The tab you are using right now to read this lab.
    2. The Jupyter launcher tab.

Go to the Jupyter Launcher tab.  Click the *Terminal* tile at the bottom left and the Launcher tab will be replaced with a terminal window that you will use to enter a small number of commands from this lab that require interactive user input.

![Choose Terminal](images/opennewterminaliniksd.png)

When you have done that, come back here to continue with the instructions.

---

### Step 1.2 Use the terminal tab to log in to IBM Cloud

Navigate to your terminal tab, and from there, enter the command `ibmcloud login -r YOUR-REGION --sso`, substituting one of the following regions for YOUR-REGION:  `us-east`, `ca-tor`, `br-sau`, `eu-gb`, or `jp-tok` which are for United States (East), Toronto, Canda, Sao Paulo, Brazil, London, England, and Tokyo,Japan respectively.  

For your convenience the proper command for each region is listed. From the five commands listed below, copy the one applicable for your region, and paste it into the terminal tab and run the command:

`ibmcloud login -r us-east --sso`

`ibmcloud login -r ca-tor --sso`

`ibmcloud login -r br-sao --sso`

`ibmcloud login -r eu-gb --sso`

`ibmcloud login -r jp-tok --sso`

---

### Step 1.3 Follow the prompts to complete the login

Follow the instructions to log in to your IBM Cloud account.  The login process from the terminal should be like this:

You will first be given a *Y/n* prompt to open the URL to get a one-time passcode, reply *n*. The reason to reply *n* is because in your lab environment your server does not have graphics capabilities and cannot open the URL.

In order to overcome this, you must copy the URL given in the command output and paste it into another browser tab or window. The goal is to obtain a temporary one-time code for you.
In order to get this code to use to log in with the CLI, you may need to log in to the IBM Cloud Web UI with your browser. If so, follow any necessary login prompts. You should eventually be given a page with a one-time code.  Click on that code to put it in your clipboard, and paste it at the input prompt back in your terminal tab.

When successful, your terminal window will look like this, but with your account information of course: 

```

API endpoint:      https://cloud.ibm.com
Region:            us-east
User:              silliman@us.ibm.com
Account:           Barry Silliman's Account (1e963a246cc69a44df65e277e14239d5) <-> 1996902
Resource group:    No resource group targeted, use 'ibmcloud target -g RESOURCE_GROUP'
CF API endpoint:
Org:
Space:
```

When that is complete, come back to this tab and continue in this notebook. Leave the terminal tab open, you will need it later.

---

---

### Step 1.4 Save your region into a variable

Save the region you used above such as us-east into an environment variable for later use by double-clicking on the code cell below and filling in your region and then running it. For example, before running if you are using the us-east region your code cell would look like `%set_env region=us-east`

In [ ]:
%set_env region=

### Step 1.5 Target the default resource group

---

#### Jupyter Notebook tip

The text you are reading now is in a type of Jupyter notebook cell called _markdown_.  It is intended for instructions, and includes text and possibly images.

Commands that you will enter (with the exception of the small number of commands, such as the `ibmcloud login -r YOUR-REGION --sso` above, that are run in the terminal tab) will be in a second type of Jupyter notebook cell called _code_.  At the top of this page you can see a dropdown box that will show you whether or not you are in a markdown cell or a code cell. The below screen snippet shows where this dropdown box is located:

![Choose Terminal](images/celltypeindication.png)

---

You can see from the output from _Step 1.3_ that no _resource group_ is targeted when you log in. The command in the code cell below will target the _default_ resource group. Resource groups are logical buckets that you can define in order to categorize your IBM Cloud resources as you wish.  We will simply use the _default_ resource group which exists already.

Click on the code cell below that contains the command `ibmcloud target -g default`. You should see _code_ in the dropdown box at the top. 

Now that you have the code cell below highlighted, click the _Run_ button from the menu above—it is the triangle icon—and that command will be run, and its output will be shown.

This will target your _default_ resource group. See the debugging tip below the code cell if this command does not work for you.

In [ ]:
! ibmcloud target -g default

#### Debugging tip

Some IBM Cloud users may not have a _default_ group but instead have a _Default_ group, with an uppercase _D_.  If the above command fails, try using *Default* instead of *default*.  You can do that by just clicking in the code cell above and changing the _d_ to _D_ in default.  Then run the code cell again. If that still doesn't work, ask an instructor for help.

---

### A note about running Jupyter Notebook code cells

Observe that to the left of the code cells are square brackets with empty contents. When you run a code cell, an asterisk (*\**) appears between the square brackets.  (If the command runs fast enough you may not even notice the appearance of the asterisk). When the command finishes, a number appears between the square brackets.  During this lab, some of the commands may take several seconds to run, but no command in this lab should take too long. Each time that you run a code cell, ensure that you wait for that code cell to complete execution before continuing with the lab. If the asterisk remains in between the square brackets for more than thirty seconds, ask an instructor for help—none of the commands in this lab should take that long.

The number shown between the square brackets upon command completion is incremented by one each time you run a code cell.  The numbers are helpful in ensuring that you have run each code cell in the correct order.

---

## Create an ssh key for the lab and add it to your GitHub account

### Step 2.1 Create an ssh key pair in your lab system

As part of this lab, you will need to create a key pair that you will then define in your personal GitHub account settings so that this key pair can be used to clone GitHub repositories.

**Note:** If you are an experienced GitHub user and already have ssh keys defined to your GitHub account, you should still create a new key for this lab and remove it from your account when the lab is over, for security reasons. **What is the security concern?** The lab instructors have access to your lab system in order to be able to help with debugging.  If the lab instructors were malicious, they could copy your ssh key, including the private key portion of the key pair.  They are not malicious—they are well-intentioned and kind. But to be completely secure, you should create a new key per the instructions in this step, and then remove it from your GitHub account when you have completed the lab. (The cleanup section of this lab helps you with the key removal from your GitHub account).

Run the code cell below to create a key pair in your lab workspace.

In [ ]:
! ssh-keygen -t rsa -b 4096 -f '/home/jovyan/.ssh/id_rsa' -N ''

---

### Step 2.2 Copy your public key to the clipboard and add it to your GitHub account

The above command created a key pair (a private part that you keep secret and a public part that can be shared safely with the world).  You need to add the public part to your GitHub account settings, so that this key pair can be used to do a clone of the GitHub repository of the sample secure bitcoin wallet application from within the Secure Build Server container.

Run the code cell below to list the public key, and copy this public key into your clipboard:

In [ ]:
! cat ~/.ssh/id_rsa.pub

Copy all of the output from the above `cat` command into the clipboard, and go to [this link](https://github.com/settings/ssh/new) to add this key to your GitHub account. When you get to that page (you may have to log in to GitHub first), put anything you want in the _Title_ field and paste the above output (which is your public key) into the _Key_ field and then click *Add SSH key*.  

**Note:** if you use a password manager and need to use copy and paste to log in to your GitHub account, go ahead and log in and then copy and paste the above output.

---

### Step 2.3 Add the GitHub public keys to your _known\_hosts_ file

Run the following code cell which gathers the public SSH keys of github.com and puts them in your lab workspace's _known\_hosts_ file. This is done only to eliminate a prompt about trusting the github.com host in the code cell after this one.

In [ ]:
! ssh-keyscan -H github.com >> "${HOME}/.ssh/known_hosts"

---

### Step 2.4 Verify you have added the key to your GitHub account correctly

Run the code cell below to verify that you have correctly added your SSH key pair to your GitHub account.  You should receive a message that says _Hi \<your GitHub username\>! You've successfully authenticated, but GitHub does not provide shell access._  If you received this message, continue in the lab.  If not, ask an instructor for help.  

In [ ]:
! ssh -T git@github.com

---

## Create a Docker Hub account if you do not already have one

You will need a Docker Hub account in order to do this lab, because the secure image that you build will be stored in Docker Hub under your account.

### Step 3.1 Sign up for a free Docker Hub account if necessary

If you already have a Docker Hub account, you can skip this step.

Sign up for a free Docker Hub account by going to [https://hub.docker.com/signup](https://hub.docker.com/signup) and following the instuctions.


---

## Create a Docker Access Token

In order to store the image that you will build into your Docker Hub account, you will need a Docker Access Token. This section will guide you through this process.

You could use your Docker Hub account password instead of a Docker Access Token, but this is not a good practice. By providing a Docker Access Token that you create just for this lab, you can delete this token at the end of the lab- thus you will not have to change your password to remain secure.

### Step 4.1 Log in to your Docker Hub account

If you are not yet logged in to your Docker Hub account, click [https://hub.docker.com](https://hub.docker.com) and click the **Sign In** link.
Follow any instructions or prompts necessary to complete the signin process.

### Step 4.2 Create a Docker Access Token

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#create-an-access-token) to create a new Docker Access Token.
Save the value of your access token in a safe place- you will not be able to retrieve the value later.



---

## Create your Secure Build Server HPVS for VPC instance

### Step 5.1 Download the Secure Build command line interface (CLI):

In order to set up your own Secure Build Server within an HPVS instance, you will use a command line interface (CLI) that is provided by IBM.  This CLI is implemented in Python and is publicly available (including source code) in a GitHub repository, which you will download now into your lab environment when you run the following code cell:

In [ ]:
! git clone git@github.com:ibm-hyper-protect/secure-build-cli.git

---

### Step 5.2 Change the working directory for the remainder of the lab

Up to now, your notebook has been running from the _/home/jovyan/labdir_ directory. For the remainder of the lab, we want to work from the directory that was just created by the _git clone_ command from the prior code cell.  Our new desired working directory is a subdirectory of our current working directory.

Run the next code cell in order to change directories to the _secure-build-cli_ directory that was created by the `git clone` command that you just ran.

In [ ]:
%cd secure-build-cli

---

### Step 5.3 Install Python packages required by the Secure Build CLI

There are some python packages required by the Secure Build CLI, and running the following code cell will install these packages using *pip3*, the Python 3.x-based package installer:

In [ ]:
! pip3 install -r requirements.txt

---

### Step 5.4 Create the Secure Build Server workload section

Part of the Hyper Protect Virtual Server for VPC process involves creating a contract to deploy the workload running under Secure Execution for Linux. The Secure Build Server has been created and distributed by IBM who provides the ready-made encrypted contract for your use.

This file is posted publicly on the public documentation page for IBM Cloud Hyper Protect Virtual Servers for VPC, but the Secure Build CLI repository on GitHub does not contain it.  That's why you are creating it now.  This file is encrypted, and only the Hyper Protect Virtual Servers key located on the IBM Z can decrypt it.  IBM Cloud administrators cannot access your secure build server at all, and you can only access it through the Secure Build CLI, which limits your access to specific functions. Therefore, by design, the decrypted contents of this file cannot be obtained by any human.

**Note:** The code cell below will not produce any output, because it copies the encrypted message into a file.  But you know the command ended if a sequence number is placed within the square brackets to the left of the code cell. 

In [ ]:
%%bash
cat <<EOF > workload.yaml
hyper-protect-basic.UxkAbpIPPLZB2LNybZ8PedWbaJkMzJz/taoVf/ybKySkgpJ55GeHkNgQPeoLEJUPG54wLyRJ4AIqn96qQDAmoiPqIy8e9ucV3MCUu0Ed5bphbJjWzDvCFOtHh7VutLVZbMdzDDMfCblkc2kJ8uVbXUdY1cktwqnM2FnNwJXX98xlNfljtc69JnVhAcUWpDCNYwALjcxyHcKWxeKRetDrMnCo66rMN7W85DnExUM17wjNh1WVw1w2UTURwApnKRtbDjJ2xkYDyovJqyJiWlBeeX1WWEOb014ft/XacbGjWuA7WZbStggxrR9U2dO06HnmvgrpwpGhlSHOT3j7BJaSz0me+ZBCvdmLDDZozbXynn1JRK6WIgR/aMdO7lCkteh6PfOES/vjhFCTQn8Yw+Padu9E3Ye8yQweqsXnpB1STL5TAuZk11y4Lhs4xvqrXHgCDuek1SSIe6wdV4UoEXjVZfBypxEvXjMfYkz8N2WmPyoRB3VW18YuPEk+RSe6tQyeur/jc0rNMQDeYdXUn1Aao0xXnQj7NO7b73qBJZfRyvgKFxib94CCyazjykmzRNKZPyci2SGkmNCKAnk0V+o7GqrO78IV1rvmwS2GTqKKSTh35ovDkXAijn/7+kWX+Ka7Jc888AnRapy45fy41LXQ4nxkVk+jELegPkWBwK4lCDY=.U2FsdGVkX19xkuUH3lUtrPHALCFCRnp68NI+RbzK44H4e0z91AAlbRAIJmDBKsi3CwIP2mbPMVhE0okoOR4Ni0Xi5KMLMV9ODBMu8JvkLCnhVosKbviqK2wTkKSi9mbNAOyj8qoS5Ejtw+HFd/X+/RWaNYZbgsSoRfEdlft1nP9luT0HSCSCGQrhwTw1M8A77D4UWK4UxHosIqTWPMh+dZrRROTVpRLDiOeKYTIoBiKxV+iD00r8wJ9BhrxLxeb5Sm0oh6iRk3ffbJhN++Kmy3g+YY7gLfoy6qXIBn/quxiW0efkSZsiR3dyBp5yK9hVcLPPxrcwXaE/gQh0XNCLjpFUe7gMdtAx4SYfjbLXGqClLd0j8UX5fHsmxQkUz3ZfE4qkWQVc/5nySzq3erDKUJBwtO+S68bl5BU/8d4JjZ/qJQ5WeJCs2nlPOcBQcknb1hWzzorrwqAbLNKwo4qTpd0cKRLlZYvLez4bJrB+A/zvcYd6/pjJysEu+fvN7iBduN4S3K670p2Mk8R5P9gprGWquMj3GubV/u6graX/7Rq8agFQfYU3MKtIxYkwtRs2eR+Q9tARiZNLAE0pVh7L7yS9zEG+HkbHBDyiUxsS1MmXhxZoA4pVy5EkczrY0wH3GtZDlXAG4QLRZtzDlVzlSxoUb9hsYSPEXDuxEc27Q/Q94O15843xop6cH2tXP4QnDq8pLtxTjisMhKbZT1FsgAFWqBRQQGQYfgwiwJVx6e3YvRiqGirq69rjj2UrDj1+85SoJoglkZeqCqmurD3MWmbUWToPaO5KfU1pZVDym6h3S207Fsj/kT8xHbmiPnkshuCPRm7jy9nuLSH7u2L7NVhlVNvMGyS5QYffx6VEVk178ZoSnsoBX0MepMhzsri0ft1rfYxJby1Pt6ax0cxzSWM44XDxzYQV38/H0Cr22BQ1gLc73gQhQ53GetJSEcbVjYWKq5eeIy6sSQJCFazgLRPMvt9/4NdlJZEgaUr6L21K1xQEdX9GpWUlLfdCYHT12w7l3X2+gJQ3CMuIXLuevfGcYhEdA6nAgsAN9QcUKFIapQdQ6mC8H98Io6RKyKl7/nb/kIlzuovLxhdTmTBrPW5X/CGxoQ4j+uVrjCt8fZLQCLurC75D5YQr5bfRErhsWLpCnchP4Q3gPZh9O8zG/TAjHsoHpB8HA4d2zSXzAscB2p9w/as1m37GOHGF8lm2QAUSjJq22/MqAQ6OBwSrSwx2HOLbgxL+eam/2HLinIuIYk55V7mA0ZGdjj3LZtFXJ3hVw8rb0jhizq18G7nDoG6u1tClwQLHzCEAYvlpaCEuOOkEX9Km2dyKK/f9Cs/NmWgNfvRJdHB8iUNuYDD2b9ZdOPgn94ewAY+cuKF6sNj0Xvl5+w93Aij2YzmRZbwjyLZmnJtAipfFIyQw7WrHC0SPsUK2L8x1v9j0Hb3lRas25LWtK1apMo3RuGl2alvh9jFxZOi9MUn5VhconFAlfGoou+0LmJN91dq43NLyiutGj1tMu5EOyWMTh2Vokf4+k0017sj4u9DEm4RafFUeAd9wLVS1sXOyWYV+W7R+bhu5GtLQsTLwYYHCbVIjOClWIqY7/dZHmAhsIHJTLV5ozXCZ87NR+Umoh0s2Kw1MP70FfWM2W15VIVsMR009+ytxqI98HM3/EYeKhropTsH887arUkqqT7+SwUYtql3AQr0lGNpU71erT6IWiL8QeSLKUZkTFrnmqDqwAs3JA//ZGdns3jgUJNwh52rfJx8D6X7ZkkcRpieeTIHzYNMjYutjeHp4JqvqoDjfsPFG7rpdL1WXy6r7EesUZuouCf+xKsHg1hPlR/mpzUZW80SuAndotCFXAJGeug9xdtV1oPUfVTAgxef6brUjA8rBLx6YJ6X6EbqEou1lzr8CCAOURRQ38Gr4lvrphz7LpVNFFsu4qj8xhFGJz1c+tbh++7nDnpiyMJu+PpwVWP+sxkGgv3Jh9LIt6Jm8hd1fx7LfY+ssHxUwsSQt0ZV2iwq9RAH8E/AWKo70SCruENxCXXO6EOXG/6BT9G1Y+PNwc+ByaFferP5xAW5ND8fpXaL5rXnDcwh9h9nqbq0QuSW1+luyl+e4Q22aMSeBQRIWhs9c9SQhX0rgcr4xAJU7VhpnubixB0o7CpuBMLbLTIYZnznDaAqC1oPe2MdNfUAPGsMbojq3MCzRquCsG8l3LK+Twr+3H8tEwQg+aeUqzCpqknvIIFx8UP9F4EDJ3F7PEYgGuAo514UHdV+eEyGQ4xetuF1H7ZPAIzIkLLY+mkkBVvimrcc8JS0HaMyKkSgjbVmNaGkOlpYUlhtH6/U8Hh+6bpsIL3z7F199jKPqvoEnbT5PvyaDCLzp77SeXjHAFfZ8AYLemb+SpHRpfa8+adHqEHjvwekysBbDw3CtOE4bjs7txpzr0RYPOU5Lb5Cz1Nn9t+tE8JjhsZYqIMDM4R0tUqxzLvGKqeW1+RAlJ+T0G3JZDmdPOOVka9IgMZBFmZ7ob4qxp/w3CoJbjCp5iAsldslj9f3KRZ2zpguCc/nWpT8g0DZFGxNzjpgR1LIyNwTZ50decjaMV6mbAcCHuSg0OPvJGj0qeRkkATUVi4Vq1fR6ic4vDuzyApRNx0WX74BE5fXhLlH5POMNFVmIsdHfbFOsH5bmJzdURyy7UcGHilbPzat+i7fRtdCy0ZaHs4ujAKBziFrWwlMvhnqvazqRKZ/k8V1VtVoeXKenKpoqWn7EBAonfDxV8BOUqTp5KinDCeKvl28miNebZl9QHP0tiad+wFFcZ7WBFh/EnQhvvhZz2kwt3Jx2E9kzXCWYhZYgO9nU0PFD68viOj+Qdsna4W1JZ61RqLTzlJ+OIIHfEB4xvD8jtfhbO/CYE+kg1wjN4OgVWpObO9O0iy2l8QFtLR9mUMqzt2cOeCeMOSIpYUa0/FQXaOEoFhBT6Scyd+BDrz6eT3AVcxc+ZR70cmzPmv2ibPI0VmaIFpUpn8XVfCMBzQ+L+hZJSOd6QNDjjshOwtQEHWi19wrzIFQ5BSiBNEPd81UWFuzZOuI13huHD++xfi0ahCpZlsNvbsSk9msGPm9I2lFwSdY1yKpIn+kGp2oxjDTYA3uWp33m+zwz7Y4RSK6XCvlXYQ/u4kxMooA+AHQstvVIK2H8tXKPOl7zs/+YgiBSI1Lb2R+p/ajp9GujsEPaoN/GZSvQNCgsDmTSbGfT2c/oyjhVf49LV/ux9opPx/Z7dRLf7pKExqf1ehauwaNp7378xNXqbB6xs/H/nCIY8fKyNuXCVPD+fHA+Roqav5MZxUpIZQ9lGxaSn4hZl3gLbbR6RZ9HcmdO8E/IM40qvVjSoKYMDzJBcvlkyzvsZQzLU4T48DfIZrWPXEyL+EceWV48PeyeouSjjoMTdp3UY+tzEWW/rUSke2VzqgVhocPnbRFJ0SdPJcNra3FOZNgp4R7b8aeuOXFqlOHTPvSJ+ZHtIRgmkgrjjUahamwUasEFuReWqhsOmdOajLADj3Idj9us8E7kAwh7rXA6j2EFhLaI0EC1xBP0IOrk38cmFXN/Viy64HP+m9FupJov3leEpKPPKUKrWcWjCcjo9tEsAJdexUWewWDoy2wUQpBSCPr66GeOHr1Ns3e3dkAtk3BLjtwtYbFTI3hi6oO6LqiUqVd0avYI3WyBe2Jd0spFv/IskVC+VCRSQ8s0d6aMwaXvgiZTukcboKgbKuB0oPaOUxwtqXVAl+NMkBOoTodFrLiqZwjTzW1/E43cwwGepc489IRQYv8vqCyOPpqS/nggnLoNtGYTZFGZndMF4RPEnW0xZ+yoaUG1Fz/cVP9kCGxLi3wfaXhYHqTt5Jbyp2ik7iz5gVkVpYlmvZRRj1GcxXPD+Ar6SmhlCConlphL4PWpEB2pCZwJ5ZbtkFAJP7ApyqPxVytpVUPIJJx26HZzzpCZ1H1Llj0bg3RZxICScAKI22+9rtSndUmkQ8GWOgBMYk6dH8PAakmoQ9euXgmK2bJJ2O6ZDDM8EQjrJLpevvNKjAAxVYZ5SPFd3grrNEVvxlNjRC3Qd2Qd6ogZE18D5o/arC59t25WPlu1KO0FolZEffbX5lHJmjqwPZb0Jkx2K/DbncdV4g3mGFs8sNnFXfKlMMKswpVjzZ/FMTKRD0m1nWAMq7ysLqv2WlkA6CMTPtklZCQ/rtqBI4NeJF35NAK6ENJ/p0NvgOKOG3h/Jf7vnhprU83/k2WEA7JTUgwwWJwjk+i8pQkuKyOsWt/wd1wC2VbieVhdgt+7FnCLodfuTd3vPLiwmugczrhb8H0ghajzJcCtAD0aN5lw5/LWWPO6pO2bClo3wGDPmOlWcM7aX5x3AoecKjmNwO3A+ncFWQ7uM3J0FLoTwmnwDBrLxp3gmgbP/pL8aqk+k/EQF7YRoWRI+MYaSEQhmfnzFviB3aWySP2IUhstQFa/ENVqZW2+MhE3uZmssfjG28rGDKrRC578DbB7u6JxRVLcKxd16E85wcgebRvzODNf4C/E6TcO7w08a1CjA1KSRznPmpIHPpa7CP93hIrumHm85+VxED1w26Lmf4OrjTvMMO+GZN7+2Efke8ZWd+kpIf+6hAb7zcsxxvOVfcaYxA76LIAPkqc02tm1blb1kWivBQ+Lj6SD2IZfbvSEpuOhH0s9WZJjrF0DhxLxko99dySQfD0pnRc1Y8byFedEXjuI/5bHhwAB2wgu7DUps2a96hQgAtCKOAhev5/tATY1giKmliutHjSY+GxQpjBjPIikffEcahT8wlpRKsUALu+GOlrI52t7i7INwtgA/tpgaVWWNlzrSm3d5faHs9k5yJE5ZoVuiOPNdbHU9jgLJcabyrZZE5PjIRIQim4ROGL/k2k5TG3mKfA3A3y/hWwY8GuNnu5tGIX7vGoEJ4TUQp7uI87/1KeNlJIIfVoBKlVcEmKVYnr/TUoE5nWmcTYG7vj4Kre5P47TE7bF4F9lKDgJsIj0y2Qsg91m5B1iWJibN1lEpxTAehcuX0ljHM0C3VDQnSB5hPZUBuLZzxAqepjhbgfek6tz/27zLDv/kOfuSa7LtS+gpWohTN4oByauaA+YQTGo0eyvN2cOdCMcvq/ytCv1FKZgSm9kzAWoCy17NeKlMCbZte+xEw6n25t2LsAiGTspOv7unOV3IXGZOUDX8NC4mhfNBTHrtApbS13d+DVwFtVNqvQb56Fpz1FSUMU+Jog2u9z37iSah6eV98kEyobPaC8QmmpcvLID1EQV6rmhTHlTSLFezYxvra+kpKxpvIrp85hUNCggNNFBQqcC7ZulApjdM0caRf846fblQIPQ7fqy1+D00JLFD0J2TbpV/eUxDeSDsgDcyXQki6MWr8i0CFdDIN/stOvUwsMAfoYRVkEpcgRNetwEd1lCW6EZAsgj/npt1HTv65vPMYvYBvdJPIq9LXxfWWLXsPknnnQMlH0zI1B+uCEeAznz7O3clSRaJzgH/t/3Qjnt5eFBC7elmC/GvFPSSe62hRcQm/E7oestVt1vxfewS1jikU/TWJOTEskxv1X7qZUXFrU7V0INx5PBRid3cXWXi2erQuyRrW7+FCf4C30unWQ1oX9Nt94evWJ9yVAs2+XqGXz1XzZV9Tjg2/xul/HO8bN5o8AwcPs/H54w+LqszI0DlktXdMfw2JkTRvn/zfaahPum0cB8Dlce7QULuym96Prlol2iQDCGlTAEJR3UScoL3BfKHW7vTNJjSEL4ZBqhn9/6BHjpLw45ag8VJqN9MSdLtpuLW09a2BdJydw+He0MmM+RdinUyfJv3n00qhRKN+LkCSyFgD4yup7zgtkzcilp5W1pClztXMXMSOa+Azey3JOKChPxIp0nFHllcqVlCr95DqQeE+IuE1egEvCY3pZu0j4mWls05eAsT8I/lcSB/xFq42MjrYQqkJ2dHwEX8KS5A2N7hkq2Ny/PF7HkOFS0Y0DYjQd5X5u6zLXn1KZpAiLF9NdFFREsJEcvQj39Z6VbnPdN5rvLS6B5Zsej3qr+GpUynK038/8SkXrCXuk/HRlV7YWFpIamWJ0Kko5sTokLIaau4vNetdUlW3InpXBEo2BL7efkuKRfyCgQ7QjDfODIDDFIPnWcg0xP8EEhi9PNxwn2BagEZKDH8I0f+5CNTetTWnuAe/mOFoUecVHZhRUeQisf0Fpu62fG1NGHGDKOxu+dcouUmH0CDuZv80HjQhqRo8pPAIBUd21VH7GLjMn3+j6IaVQphuvS98qI0aGMm9RAy4enPy7VUUSMmGbw654fIqUVIvJFlRRFvLNoAdO6Xa8q2C5bIQtRtg3+R9JRXd40KADtW4Ql/CWZBQirxxgVLgtaPSEwKM8Opuz72Kou+hT+4ctYLhfbOOCRnkRSXoiB2RpbNLVtRx/rI5IRs9z8OfPuYzwyhsuta2VkkU0BSaFxJFi8jeykW8gZqX343f5S2cONlPoWFYVmZ/aGsMP9q/UXmgUazzb/vtDAgjSsKpT0SNofnYkznz0E5EkC7coHqSe3efH5NVLfopPyAsoE1anAtF9KuQk2AWQN68OCsNahMavWFg8m2oBoF+1o0yIsBUVfMM/91l29lpFcKL37gVJVFAcQJhFEd5JNFd7kKv/fukl+WEF1gNXrnhFEb/ckC04QbqTGVlAfBL2K9K+I8rcGfzgYhG9N3Q8+sTMzFqwgmBUoycTHZyAIIQDhKjgp43FRsJqJlDFWqmRfnJqYaiI5/zjZ4BUdlMJZJ9Ku2asFij/lBf7i1XKk0R8T2RuINSkoyGdHOwOC9d3niVxm3P2zkpIaF6oQuKcO7+pEprl2jteKay83pMS5qY9LrSDfHZM04lt9Db0n57HbjyOnE5TMaSr4EkD9qJMuQ6fo1GH0KffRQgyy1kp3owwgekdIihDPVWIkqX6p+4snepzmUpwwiy7MpGCmH2oFpoUDWT+FahqjCACBruE4KS//9MnydJE6SFOfNiSwh43doWLZs/G2e6F79bLmZ6asg19VGr1ja2E4HgxUN4kWCiPBLshgiU0M5SIpgNKaGf5DE9pNhUwDo1r3Rujnf1hWlFQBeoDmvuqCDSY6kryMDMtN/DCNchI6J/3K57RlnJKwsDM/mDNaKxI2pKQkS74dpKtjYDgpi9IhgNpeM3Y9xu7TdeJgFSHbqLPprpmzU4V5UP2njg4T6Au+rGhFvP97DcC+zIcZcpCgjYNz8kjmnTr0ZNYnCBwu1939jhn0Qq+Gc7mwQS/Dqxj+p4NcIte8KK3qdLTied5j5sB+XLD+OBRccvpnDW9e1ltv6ND+Q0JyECCOuqBsME57+xYyUKKlQjFe+iEVLrEtC78TXjsGiPkdSKeo+HNt+gkwJEANkvKGMu6TrkSoHuxb84lyfRS10LskCNGMbnw4Rf8RcGRIuKNJZm3+YYgJcm0N0uLIwFBDbdgUXAwyPmq0CYTw+8rwPi1O3ylug/LGzpW80K9FmlxfDY8ZLuDSeJ4sja0j6wIuZdMY5CE79E5SouJSAJWZRfbEwxPSwuET2R8hI2ICoN487RfG86Qr/VBUBA50uPKXRh5fUsAOlaKUfIbKxpbeyNbxqWK459Nd/KoZEfHvLuwUYdO2RZZauBpFVaSpC3XKZr+eO9hYWp+MAgBdoUNsIO0Zbxp7XegWg0zEl8z9sVlLxSK4U0Vo+Lxy8fMquJM3lrSSs2rwjiRKNbGffg4jV24jph5bqV4TzUfIaBc/HEZYLj0/9MAIC/JdZasBvZEegsO+qUmjLwGmXKTVPUAIZlYC2o4/TK0X5JwmTWRoC0PFNcT/x0r9LzP9Rfoy7NDZC4BQgAA42cupElIEQNLtSLo2ZLPVd+9+EW9dNfX+AsWZiLvTVS06zxZf6YZvb6XZYnkS+bJq9soOQ6KCRxehYTzvJdv3s06X4QHfrY8PdVMoxqPXqe2RPMQsOmNodoWv3MZha0Bjkhl2g7hRl8colS0fTvmQcR/ES6crYyw6BTANXTkCfWLBHmznkJPtCeFi0hCJ5z2WuCQFM7OUikkeH/W4HG+Rg3bzhsqluyqW/gWXNYQ/tcHCLZHgyG4K0EIv/f/6U9PLKx6vxekkFWxBIo0+0FKk64mABPH6pnmpmowHcPtaMAlG2CAnSzY578xZnq+2lx1LuyfH/M8SivD2nTRPZdGyv2Ir4Dsgu4MWno2PwvY8w76+WgsqW1DHYJzo9mJK5VfjOM5wv6fmqDw76a09xvO5JDQhgMZ+Ai7pxlYsYSHyjcYUrReVZjmmt+bYXNzQh+3dJQisOJqPAj1G3j8+B6sIpdy6+OHk9pF839viYgogw+ZoL0B9KEk8YP1S2jh5PjaYLb65/c6uG/F3QnXGLADE8xU3B1tbTJHTLr605+TKcVMrBx0eSTWOdOXmx0LGWkHtEJJuJ7wo1b3D5d6Mjk8rlwEqpfD/8BRV541qmyBnDXNnY9ndvRxFafP8sUrssl4ljgruoYS3pAvJhy81Cs0MFg59j+kEipL5SRJ0cuFIf7NyjO6Ktx0xYTGL04JYh3hmPrc9nYv+jrzZqWSH82AdrbQqQBk7erb1dZLYgF58slTf1XlbpOuo3txACQ+Pj3blO14o4Mpn1O/Oh6oEKn8BTKyahjRGBKeDV1m8tttBM4Eb7yggjeidEYqJvilc5eEI6KDpzjn+Z+iicAWklrCnuuZ2+Zh/jOg0t4YeN4zEvZrZCgsAqPZvc/GLy7n9aTrh0kjiP63NMHSzFyumTcoCYSm85RjCTz5pqqW2QWS0DNomgpmRJI33+OVSDbXBUwRa33lPcRXNe64RRZYL58bdVCffQ00BTm2JK4QH3m5Qdw1UCptEhqQBWXOmt7mYnOMP/U9SiRygQ02JpcoUAo9CsonAnV9TKX+ae/AYOdJAjMyswKsxj4CmfoT+OmBgv37z92/Vl01sGSyeeM/tMISVgfSPzi3eXp83x2seEMX6hDSkyzcyqMaRFyNuhJwcvzwM2b1SrdyzfQjRe98NLWW3DzJvQxhOYhJKXgPYnrUotGTPQodQViIjLFHfDNAfPhMOk+xwh+zNEQ1lAyekbcN5YyY8XNCXYQpr+d/FdNUNkN5LjOjvvgvcBguSOJYmCbDiM/WXmt1XxBaMWLatI8r7ceKh7TW9zmMN5XoWXjTBJQ6d1I0/lkksp4HOBj+QV2x2kC8McCRDRmPM4Ft8GhTCGiLH2cF1TjCgy4X10l2DXiujsIdagNehr36MWU9q98TjmFcpZaoHsFUpiq6LlpyuMV+tHRGQmC2oJ0g/X2rRx4tzhc2DUUSRtxKB/7iGNPS3N+9A/0ja9JwxF0cCE5uL0OoABFWF9G7BOsX6FQaBcC3zLEzDZWjjsw7ILbcOQAbsRawguayj4FuGDkM69eJrMI8kBzmdD21MoUGDKcUtAttZph2MSnhIsmszBEKarHM3xT8tF+6mw5/5sj/6BsGW3mUh7l2KLunIIpKkwsaKShiLmKmhqaxJid3ZTvB7rc884/wQJWFKdB6fSo1dPcQ2V6e5x3DZLt6SeYbjqQGGNJmfGQXSTg1nJfBVYttH7SFALQcZ1XAZzTBBKQEGs6l3KW95HlcRDlXfjCr461WXGISJ6dKCJPRJgtO5Zn33pUVyxBfl+zYqrL8t7bylzIzfuxfAJrglxfXzwnRULGvunLaarXjOmt3bMAgWVjr9xakxXl+53zVoEmjH+kdeCuYh5BMbtScg1/v855XFm/NfabaYvR81vl1UoDdw9sgLJtP+gZaSDGvkTVnW9/HvNAEbpJylmh3IBBoc2OTJnEOqOtKt529TXRD+K272rowbs539VbpX3GTThqVmAAm/h1J+kmTgRknD76w8QPZWd/lqRuhahwGm4FcnC6EktvZkiag89U5MfXm/Ni2n7cvEOyWivEz3J5ksjvoaFcUUI5HiNi+yYKvBNSpqJFf7W3scI7hDyOv7ykTO9vHK+foEflcL9szoDpZcU6OzfvkGULwe7SJdvZt6u/z3jDmv/c5WlrxlBStTAx79Y9sJxCO/EegsiWqk7gZEjGFjSW1NnAtzNxYL0ThtnMcbF1femxNZq5UxNw/jqgbUlglQIMpOYFWN8nlPQILfFMr0StOKLr5Nh8dJQzC4J819VkFCp2M+Qp/K78rhA8dmdwS9Z1Pui1iWy7eRQZPE9JqViFJDR+SAjOVWp3eO46MDbTRvhunwHN2Mw2/Ux332GnVFKZQcc6nwaCxFNm7eopHeTnIE+kAUC4er2zzlAjiZdwvPG4D74K/xz/55Eqz6sYq4MT4HdoFMEwD7qnOFo8jkB/UwWN4cnXobCDH+xWSse7aXCUaDbuj8ULEiMXH0SHt484HvG9Pkddi1UtJt7jYVKkYTopVbUzhxq0FAKdBa2ijAaLr4YbvNW9LlMNnKqNzerSx6OKN8NnL8VDZnryt41C3djc2a1P590W25kxbEXxss9M6rYmBAEMvM21MdbajvJ2dt1hlgHxI3AkSGS29aDWotpRqElJ0OyYoDMSpltB9oTsjdZYiFY4ZIibnURYnSaLp7H5AD9N0gE+49++B6kal0KBHV68OREoI9B3v8pPzV5DcgAympK6/Fxnm7c8hmUlmJyDW6KwJ1LQLc+PJha2Jnip+AMViRBTvVpdNAhVWt/kg+KdCHKQc67vlZX/Q5fh82MshZhTXKgXDC+zcORJDKp3xm6Uk6qBMVAYJvpLhJbcGre3gg9KgPuuJYr4PW0kUtwufQF7u+d5Vg2/cliO/aYnjmy6cp9UzWv+YkgQn8p2HAWDJVQw1Upu7dHOVuk9lsabnMG4I+SeebeCmXd2+eGP5wnPQF9ozZjs1aISwp2zuCWhGT++QngLcZEu7DWRKv/Lx65ZbTjig5PrwLM00mKXG/AE/P+yWXKbkESK0jjGAtUz4/n6e848YDzDccX49+KIFhAynplNP6jdfj3Mx492Q6DoM9Susjoo9r7EBBfR0Q1ilMJgaXcE2UbRjG7KRCtvHj+SOO6oo8Q6nLSD67JJhuPv/J+u8zHwRbc4L4yBtUvyznyynGN5Z5L3hUClmABh5CPxUAOPzTtXskxLlA5t8UZJRCEQw15h9Tmak0IClKga3EVTlG6Vocrl3wOQ6cYpoF4FZB1zm0V+VJlFrYEZrNQ8O/AHE3bHPu+tXkN3R4wY08OOeX9M5jtPnkdStRh6KIGjHIZLzWVDOz7HMRcLQ7IUBxxBfQ5PSo3XNWLnAIpOOopeTNA7JqaYC+Ge2FerZXkY3M409VgwZaxJ1Zm/BxkcgkdGJfRHP/Xo4zURedUjxpcOmg+k91zSLTRawUaiHY32YcH69Raag/fL5y2rYfMz8kd6mmWN/6fEqVZ+uYQXj0om773KpuulHPCgyk9t2RDFzqJUKyuGjQZ6kdPgTUPiKQjlZ1VDa+fc6FEgpuuHMVxBkxCVoUBOHyn2c8O/BHmiCEyHK3hf8CSkQHz0KZ0qvRgRCPvdh3PIjewowSnbJfUd/6xZqSh3PhDWOC8DUfApBPw//Hzf19fHY1cWAlAGrUzVtaNkKxwMVAdJwhDc1rXhN0Wv2w1AAoEMoDnrE0YXuUI4n63IlcsjvCvD6QEyDtdbQrne8PdqXVbPhVO4j/65YqkSUL1X5kTSjJvR7tiHGrVw5Bg/hehzNYsEhe0pZ5AczXwPzprmEYciU0Uxkt/7QFRWOsDmFPRuyElRxoZ/GpuCI7OssTPA1UG0NhzzsdgrtOe8QXo33FdX371dnnO/AuRrQB8mklLUh4pQREDH4k0x37bZbsqSlG1KYnde2ovYqTVXhxYgkYCcdYEGkxwpjD84LmK6dGqHcI4iBC/tlu9u65tv5ziqcF8K7tatrtWxqHT7ZJiDfgM9af154ktzo/hUVIdZ7SX8/6oaRcW/Mq/+/py0o6kivG3vZSsgOEsBF7gfCfORFMCrgwsKkBbOVVGtBj82NS8ll4aLNeG+daeGlTsgFDSU+9TeVjEWLTMR7rW5WIxjvORW9oBnLbB0rvi6T7ZZ8M6x61HyTmapwQYkNncoNYmUGHWTsdmFyTmNF1p7eToMlf00rUFxXt0w==
EOF

---

### Step 5.5 Set environment variables for your Docker Hub username and token

For your username, paste your username after the = sign of docker_username= and then run the code cell. For your token, paste your Docker Hub token in the 2nd code cell after the = sign of docker_token

In [ ]:
%set_env docker_user=

In [ ]:
%set_env docker_token=

---

### Step 5.5 Create your secure build server custom configuration

Anybody who uses the secure build feature of HPVS will use the same Secure Build Server image, but that image is deployed to that person's individual HPVS instance, and that individual instance is uniquely customized through a configuration file. This ensures that your Secure Build Server can only build the application you specify, and push it only to the registry that you specify in the configuration file which is created by the below code cell.

**Note:** This code cell also does not produce any output within the Jupyter Notebook for the same reason as the previous code cell.

In [ ]:
%%bash
cat <<EOF > sbs-config.json
{
    "HOSTNAME": "sbs.example.com",
    "CICD_PORT": "443",
    "RUNTIME_TYPE": "vpc",
    "IMAGE_TAG": "1.3.0.11",
    "CONTAINER_NAME": "SBContainer",
    "GITHUB_KEY_FILE": "~/.ssh/id_rsa",
    "GITHUB_URL": "git@github.com:ibm-wsc/secure-bitcoin-wallet.git",
    "GITHUB_BRANCH": "master",
    "IMAGE_TAG_PREFIX": "s390x-v1",
    "DOCKER_REPO": "${docker_user}/secure-bitcoin-wallet",
    "DOCKER_USER": "${docker_user}",
    "DOCKER_PASSWORD": "${docker_token}",
    "DOCKER_CONTENT_TRUST_BASE": "False",
    "DOCKER_CONTENT_TRUST_BASE_SERVER": "",
    "ENV_WHITELIST":  ["ZHSM", "APIKEY"],
    "ARG": {
    }
  }
EOF


---

### Step 5.6 Verify your custom configuration file

You can run the code cell below to display the contents of the file you just created to ensure it looks good.  If you made a typo, you can probably fix it by going back to the above code cell and running it again. 

The code cell below pipes the output of the _cat_ command to a utility called _jq_ (for _JSON query_) which expects well-formed JSON (JavaScript Object Notation) data. If you made a mistake while updating the file that causes a JSON syntax error, the _jq_ utility will detect it and print an error message.  

If you put in a wrong value that doesn't cause a JSON syntax error, _jq_ will not catch that, so inspect the output of the code cell carefully.

In [ ]:
! cat sbs-config.json | jq .

---

### Step 5.7 Generate client certification authority and client certificate

Secure communication between you and your secure build server is achieved by using mutual Transport Layer Security (TLS). TLS is the same security protocol that protects most Web traffic today, but most Web interactions only require the server to be authenticated.  Mutual TLS refers to the situation where not only does the client want to ensure that the server is legitimate, but the server also wants to authenticate the identity of the client.  Your Web browser is constantly receiving server certificates and verifying them while you surf the web. Client certificates are what are used when the server wants to verify the client.  

Create the client certificate and client certification authority (CA) by running the code cell below. You should receive two lines of output, with the second line being `INFO:root:client_certificate: generating client CA and certificate`

In [ ]:
! ./build.py create-client-cert --env sbs-config.json

---

### Step 5.8 Generate server certificate and server key

Create the server certificate and server key by running the code cell below. You should receive four lines of output, with the fourth line being `INFO:root:server_certificate: Successfully generated server certificate`

In [ ]:
! ./build.py create-server-cert --env sbs-config.json

---

### Step 5.9 Display your environment which shows your client certificate and CA and your server certificate and key

The next code cell will copy the client certificate, the client certification authority, the server certificate and the server key, all in base64-encoding to a file for later use.

If you already know about _base64-encoding_, go ahead and run the code cell.  If you don't, read the next paragraph if you're curious about what base64-encoding is.  

Base64-encoding is a means of encoding any data, including binary data, into human-readable characters.  It takes groups of 6 bits, converts these six bits into a number from 0 to 63  (2 to the 6th power is 64), and replaces these 6 bits with one of 64 human-readable characters.  Which 64 human-readable characters were chosen?  Lowercase alphabetic characters a-z for 26, uppercase alphabetic characters A-Z for 26 more, the digits 0-9 for 10 more, which gives us 62.  Then `+` and `/` were chosen to make 64!  It should be noted that base64-encoding should not be confused with encryption.  It is easy to convert a base64-encoded string back to its original format.



In [ ]:
! ./build.py instance-env --env sbs-config.json | grep -A1000 -m1 -e 'CLIENT_CRT' | ( SPACES=$'    ' ; sed "s/^/$SPACES/" ) > cert-env.env

---

### Step 5.10 Create an environment variable for the vpc

Set your vpc name into an environment variable to reuse it by running the following code block.

In [ ]:
%set_env vpc=secure-build-vpc

---

### Step 5.11 Create your vpc

This lab uses its own vpc to keep things neat and tidy for the end user. Now it's time to create that vpc and save its ID into an environment variable

In [ ]:
! ibmcloud is vpc-create $vpc

---

### Step 5.12 List the environment prefixes for your vpc

Use the vpc variable you saved before to find the address prefixes for your vpc

In [ ]:
! ibmcloud is vpc-address-prefixes $vpc 

---

### Step 5.13 Set your subnet as an environment variable
Set your subnet as an environment variable by running the following code block

In [ ]:
%set_env subnet=secure-build-subnet

---

### Step 5.14 Create your subnet

Use the address CIDR from the previous command to create a subnet for your VPC using one of the zones in your region.

For example given the following output:
    
```
Listing address prefixes of vpc r014-03b31802-8605-41a7-82c0-ce7f46ad39a7 under account IBM as user Garrett.Lee.Woodworth@ibm.com...
ID                                          Name                                CIDR block        Zone        Has subnets   Is default   Created   
r014-d1592cfb-d65b-4691-acfc-40317d366312   passover-flyable-entitle-goodness   10.241.0.0/18     us-east-1   false         true         2023-08-22T16:51:29-07:00   
r014-cdb78388-ecca-48a5-a7e1-f1f56aeae87d   pretender-deputize-pushup-cover     10.241.64.0/18    us-east-2   false         true         2023-08-22T16:51:29-07:00   
r014-64686aeb-3fb3-4cd5-997b-e30d9cb9c9e9   dropbox-unfold-elite-strategy       10.241.128.0/18   us-east-3   false         true 
```

I could create a subnet for us-east-1 with cidr block 10.241.0.0/24. Fill in the CIDR block for your zone with /18 replaced with /24 and run the commands below similar to the following example:

```
zone=us-east-1
```

```
cidr_block=10.241.0.0/24
```

In [ ]:
%set_env zone=

In [ ]:
%set_env cidr_block=

In [ ]:
! ibmcloud is subnet-create $subnet $vpc $zone --ipv4-cidr-block "$cidr_block"

---

### Step 5.15 Set your gateway as an environment variable

Set your gateway as an environment variable by running the following code block.

In [ ]:
%set_env gateway=secure-build-gateway

---

### Step 5.16 Create your public gateway

Create a public gateway to access your instance

In [ ]:
! ibmcloud is public-gateway-create $gateway $vpc $zone

---

### Step 5.16 Link your subnet to the new gateway

Link the gateway you just created to your subnet

In [ ]:
! ibmcloud is subnet-update $subnet --pgw $gateway

---

### 5.17 Save your security group ID for later use

Run a command to get your vpc's security group ID and then save it by pasting it into the code box below after the = sign and running the code box below.

In [ ]:
! ibmcloud is vpc-sg $vpc

In [ ]:
%set_env sg=

---

### Step 5.18 Open port 443 on your security group

Next, you will open port 443 to public traffic to use it to communicate with the secure build server.

In [ ]:
! ibmcloud is sg-rulec $sg inbound tcp --port-min=443 --port-max=443

---

### Step 5.19 Create Logdna Logging instance

You will need a logdna instance to use for logging from your Hyper Protect Virtual Servers for VPC. Thankfully, you can get a lite instance for free if you don't already have one. Since you can only have one lite instance, if you already have one you can reuse that one for this. However, if you don't create it with the command below:

In [ ]:
! ibmcloud resource service-instance-create secure-build-logdna logdna lite $region -p '{"private_endpoints_only": false}'

---

### Step 5.20 Retrieve your IBM Log Analaysis instance's ingestion key

Navigate to the IBM Cloud UI (signing in if necessary) via [this link](https://cloud.ibm.com/observe/logging) which will take you to your logging instances.

Click the blue **Open Dashboard** button in the upper right for the logDNA instance you are using for this lab.

Click the *Install instructions* icon (that looks like a question mark) in the lower left of your IBM Log Analysis dashboard:

<img src="images/log040.png" width="470" height="952" />

In the window that opens up, click the icon highlighted below in order to copy your IBM Log Analysis instance's ingestion key into your clipboard.  Paste it into the code cell below and run it to set an environment variable for use later in the lab.  **Treat this ingestion key with care like you would a password, _especially if you are using an already existing IBM Log Analysis instance_**. (You do have the ability to return to this screen to retrieve it later if you lose track of where you paste it).  See the below screen snippet:

<img src="images/log050.png" width="936" height="186" />

In [ ]:
%set_env logdna_ingestion_key=

---

### Step 5.21 Retrieve your IBM Log Analysis instance's host name

1. Scroll down on this same window and choose **rsyslog**
2. Use copy and paste to save the information that corresponds to the information highlighted in the screen snippet below.  (This information will not be highlighted on your screen until you select it as we did prior to taking this screenshot). You will be using this information later in the labs.  This information does not have to be kept secret.

<img src="images/log060.png" width="802" height="671" />

You do not have to follow the instructions that the popup window is giving you, you only needed to copy the information as directed here in the lab. You can click the **X** in the upper right corner of the popup window or just click outside of the popup window so that it goes away. Paste the hostname into the code cell below and then run it to set an environment variable for use later in the lab. Keep this tab open to see your logs when you launch the server (since the lite plan doesn't save logs so you have to have the instance loaded in your browser when the logs occur to see your logs.)

In [ ]:
%set_env logdna_hostname=

---
### Step 5.22 Prepare the environment section of your contract

Time to create the environment section of your contract with all of the information you collected before

In [ ]:
%%bash
cat <<EOF > env.yaml.initial
  type: env
  logging:
    logDNA:
      hostname: $logdna_hostname
      ingestionKey: $logdna_ingestion_key
      port: 6514
  auths:
    docker.io:
      username: '${docker_user}'
      password: '${docker_token}'
  volumes:
    hpsb:
      seed: "testing"
  env:
    registry: "docker.io/gmoney23/secure-docker-build"
$(cat cert-env.env)
EOF

---

### Step 5.23 View your newly created environment section

Check on the environment section of your contract to check on your work

In [ ]:
cat env.yaml.initial

----

### Step 5.24 Create a script to encrypt the environment section of the contract

Create a convenience script to automate encrypting the environment section of the contract


In [ ]:
%%bash
cat <<-EOF > short-flow.env
# Create the env section of the contract and add the contents in the env.yaml file.

#
# This specifies a file will be encrypted and signed and is the primary output of this script.  
# It is combined with the encrypted and signed workload section that is created by 
# another script (flow.signature which is one directory level higher)
# Note: this file will also wind up one directory level higher
#
ENV_INITIAL="env.yaml.initial"
ENV_PLAIN="env.yaml.plaintext"
ENV="env.yaml"

#
# This command adds the public signing key to the plaintext environment yaml.  This key is used inside 
# the Hyper Protect Container Runtime image to verify the signature over workload and environment sections of
# the contract. 
#
cat \${ENV_INITIAL} ./pubSigningKey.yaml > \${ENV_PLAIN}

# Download certificate to encrypt contract for Hyper Protect Container Runtime:
HPCR_rev=11
CONTRACT_KEY=./ibm-hyper-protect-container-runtime-1-0-s390x-\${HPCR_rev}-encrypt.crt
curl https://cloud.ibm.com/media/docs/downloads/hyper-protect-container-runtime/ibm-hyper-protect-container-runtime-1-0-s390x-\${HPCR_rev}-encrypt.crt > \${CONTRACT_KEY}

#
# This variable holds a random password:
#
PASSWORD_ENV="\$(openssl rand 32 | base64 -w0)"

#
# This variable holds the output of the command pipe that
# takes your plaintext environment yaml (\$ENV_PLAIN) and encrypts it using the password that 
# was generated above (\$PASSWORD_ENV) and then base64 encodes this encrypted environment yaml
#
# As long as nobody else knows your random password (\$PASSWORD_ENV) your data is safe.  
# But, the Hyper Protect Container Runtime has to encrypt it, so it needs your password. 
# How will it get that password securely?  Read the next set of comment lines to find out.
#
ENCRYPTED_ENV="\$(echo -n "\$PASSWORD_ENV" | base64 -d | openssl enc -aes-256-cbc -pbkdf2 -pass stdin -in "\$ENV_PLAIN" | base64 -w0)"

#
# This variable provides secure passage for your random password.  How?  
# It encrypts it with the encryption key of the Hyper Protect Container Runtime (HPCR).
# (A key that is encrypted by another key is often called a wrapped key).
# Only the HPCR image has the private key which can decrypt this. It is protected from 
# access from any administrators.  So, malicious actors cannot do anything with this
# wrapped key, even if they were able to get a hold of it.
#
ENCRYPTED_ENV_PASSWORD="\$(echo -n "\$PASSWORD_ENV" | base64 -d | openssl rsautl -encrypt -inkey \$CONTRACT_KEY -certin | base64 -w0 )"

#
# Use the following command to get the encrypted environment section of the contract:
# This variable holds the output of a concatenation of a header, "hyper-protect-basic",
# Your wrapped key, and your encrypted environment yaml.. 
#
ENV_ENCRYPTED="hyper-protect-basic.\${ENCRYPTED_ENV_PASSWORD}.\${ENCRYPTED_ENV}"
#
# The above variable writes the encrypted environment section to the directory one level above
#
echo "\$ENV_ENCRYPTED" > \$ENV
EOF

---

### Step 5.25 Create a script to generate a signing key

Create a convenience script that will generate a key to sign the contract

In [ ]:
%%bash
cat << EOF > flow.prepare
# Use the following command to generate key pair to sign the contract 
openssl genrsa -aes128 -passout pass:test1234 -out private.pem 4096
openssl rsa -in private.pem -passin pass:test1234 -pubout -out public.pem

# The following command is an example of how you can get the signing key:
key=\$(awk -vRS="\n" -vORS="\\\\\n" '1' public.pem)
# echo "  signingKey: \"\${key%\\\\n}\"" > environment/pubSigningKey.yaml
printf "%s" "  signingKey: \"\${key%\\\\n}\"" > pubSigningKey.yaml
EOF

---

### Step 5.26 Create a script to sign the encrypted contract

Create a convenience script to automate signing the encypted contract

In [ ]:
%%bash
cat << EOF > flow.signature
# combine workload and environment
cat workload.yaml env.yaml | tr -d '\n' > contract.yaml

# Sign the combination from workload and env being approved
echo \$( cat contract.yaml | openssl dgst -sha256 -sign private.pem -passin pass:test1234 | openssl enc -base64) | tr -d ' ' > signature.yaml

# Create user data and add signature:
echo "workload: \$(cat workload.yaml)
env: \$(cat env.yaml)
envWorkloadSignature: \$(cat signature.yaml)" > user_data.yaml

echo ""
echo "import `pwd`/user_data.yaml into User Data or copy and paste from below:"
echo ""

cat user_data.yaml
EOF

---

### Step 5.27 Create a script to produce the encrypted and signed contract

Create a convenience script that will create an encrypted and signed contract using the prior scripts

In [ ]:
%%bash
cat << EOF > makeContract
. ./flow.prepare
. ./short-flow.env
. ./flow.signature
EOF

---

### Step 5.28 Create the encrypted and signed contract

Use the script to create the encrypted and signed contract

In [ ]:
! . ./makeContract

---

### Step 5.29 Display your signed contract

See your signed contract that will use to deploy your secure build server

In [ ]:
! cat user_data.yaml

---

### Step 5.30 Set your instance name via a variable

Set your HPVS for VPC instance name as a variable for later use by running the below code block.

In [ ]:
%set_env instance=secure-build-server

### Step 5.31 Get the hyper protect container runtime image

Run the following code block to get the hyper protect container runtime image for your region.

In [ ]:
! ibmcloud is images | grep ibm-hyper-protect-container-runtime-1-0-s390x-11 | cut -d " " -f1 > hpcr_image.env

---

### Step 5.32 Deploy the Hyper Protect Virtual Servers for VPC Secure Build Server

Use your encrypted and signed contract to deploy a secure-execution enabled guest using Hyper Protect Virtual Servers for VPC

In [ ]:
! ibmcloud is instance-create $instance $vpc $zone bz2e-1x4 $subnet --image $(cat hpcr_image.env) --volume-attach '[{"volume": {"name":"secure-build-volume", "capacity":10, "profile": {"name": "general-purpose"},"user_tags": ["secure-build"]}}]' --user-data @user_data.yaml

---

### Step 5.33 Check the status of your Secure Build Server HPVS for VPC Instance

Check the status of your secure build instance by checking the logs of your deployment in LogDNA. 

If you have for some reason closed your LogDNA instance, navigate to the IBM Cloud UI (signing in if necessary) via [this link](https://cloud.ibm.com/observe/logging) which will take you to your logging instances. Then, click the blue **Open Dashboard** button in the upper right for the logDNA instance you are using for this lab.

Wait a few minutes for logs to appear and see the container come up in the logs before moving on to the next step.

If there are issues with your deployment such as no logs appearing after 5 minutes contact an instructor for help getting unraveled.

---

### Step 5.34 Get the network interface card (nic) ID for your instance

Get the network interface card ID for your instance and save it as a variable by entering it into the next code block after the = sign and running the code block.

In [ ]:
! ibmcloud is instance-network-interfaces $instance

In [ ]:
%set_env nic=

---

### Step 5.35 Create a dedicated IP for your instance

Create a floating IP to access your instance with using the following command.

In [ ]:
! ibmcloud is floating-ip-reserve secure-build-fip --nic $nic

---

### Step 5.36 Save your IP as a variable

Save the `Address` field of the above command's output (your floating ip address) as a variable for later use by pasting it after the = sign in the code block below and then running the code block.

In [ ]:
%set_env sb_ip=

---

### Step 5.37 Set your IP address to the hostname of your secure build container

Map your secure build container's IP to hostname in /etc/hosts

In [ ]:
! echo "$sb_ip sbs.example.com" | sudo tee -a /etc/hosts

## Securely Build the Bitcoin Wallet Application
---

### Step 6.1 Check the status of your Secure Build Server

Run the following command to check the status of your Secure Build Server instance. The value of the _status_ field in the output should be empty, i.e., `"status": ""`:

In [ ]:
! ./build.py status --env sbs-config.json

---

### Step 6.2 Initialize the configuration for your Secure Build Server instance

Run the following code cell to initialize the configuration for your Secure Build Server instance. The status of the output should be `OK`:

In [ ]:
! ./build.py init --env sbs-config.json

---

### Step 6.3 Start the build of your sample secure bitcoin wallet application image

The code cell below will start the build of your application image. The command returns quickly, as soon as it submits the request to start the build.  Your status message from the command output should say `OK: async build started`, which indicates that the build of your image has been started up on your Secure Build Server container. The command does not wait for the build to complete. The build will take about twenty minutes. Try to check out the next two steps before you take a break.

Run the below code cell to start the your build:

In [ ]:
! ./build.py build --env sbs-config.json

---

### Step 6.4 Check the status of your application image build

Check the status of your build by running the code cell below. You can check it periodically—your build should take about twenty minutes to complete. You can run this code cell periodically, as well as the command in the subsequent code cell (_Step 6.5_) which displays the log messages from the build.  For now, check the status by running the code cell below.  Your build has completed successfully once the output of the code cell has the value of *success* in the *status* field:

In [ ]:
! ./build.py status --env sbs-config.json

In the above code cell, until the build completes, you may see several different statuses, including:

_cleaned up_   This status can appear at the beginning of the build but you may not see it as the status quickly changes to _github cloned_.

_github cloned_  This status appears most of the time as this status remains until the image build is complete, and this takes the largest amount of the entire build time.

_image built_  Once the image is built, but before the push of the image into your Docker Hub account is complete, you may see this status.

_image pushed_ Once the image has been pushed to your Docker Hub account, you may see this status, but usually the build completes very soon afterwards, so you may not see this status, as it should hopefully quickly change to _success_ status.

_success_ Once the secure build process has completed successfully, you will see this status. Once you see this status, you are free to move on (Step 6.10). Remember, you can run the command to display the build log periodically (Step 6.9) while you are waiting for this _success_ status. 

---

### Step 6.5 View the log from your application image build

Your secure image build will produce a lot of output in its log. 

You can check the log of your build by running the below code cell. 

**Jupyter Notebook tip 1:** In some environments this output may be placed in a scrollable window, but in other environments all of the log output may be placed into the notebook, which makes navigating to the rest of the Notebook more difficult.  If this happens to you, you can enable scrolling by clicking in the output, then right-click and choose _Enable Scrolling for Outputs_. This will place the output into a scrollable window. 

**Jupyter Notebook tip 2:** Additionally, you can clear the output of this command from your Jupyter Notebook by clicking within the output, then right-click and choose _Clear Outputs_.  (The build logs are retained on the Secure Build Server and can be retrieved at any time).

**Jupyter Notebook tip 3:** Running a code cell clears the output from any previous execution of the code cell, so if you want to view the log more than once, you don't need to clear the output as described in the tip given in the preceding paragraph.

In [ ]:
! ./build.py log --log build --env sbs-config.json

---

### Step 6.6 Download the state image of your Secure Build Server (After build has finished)

Run the code cell below to download the state image, which is a backup of the state of your Secure Build Server instance. This can be used to recreate your Secure Build Server if necessary. This backup also contains the logs of any builds you run on your Secure Build Server.

In [ ]:
! ./build.py get-state-image --env sbs-config.json

---

### Step 6.7 Verify the manifest file from your build

The Secure Build Server creates a signed manifest file for each successful build. The manifest file is useful for audit purposes. You can verify the source and integrity of the build and the built image or you can pass the manifest file to an auditor to do so. Run the below code cell to download and verify the manifest file:

In [ ]:
! ./build.py get-manifest --env sbs-config.json  --verify-manifest

The above command downloads the manifest file and verifies the digital signature on it.  In the last line of output from the above code cell you should see *verify=OK* which indicates that the digital signature is valid and is proof that the manifest file has not been tampered with.

---

### Step 6.8 Download image digest

Download the digest of your image to use in your docker-compose file later.

In [ ]:
! ./build.py get-digest --env sbs-config.json | cut -d ":" -f2-3 | cut -d " " -f2 > sbw_image.env

---

### Step 6.9 Download signed image public key

Download the public key used to sign your image during the secure build process for later use.

In [ ]:
! ./build.py get-signed-image-publickey --env sbs-config.json

---

## Create HPVS for VPC using the image that you just built

### Step 7.1 Create a new working directory

Create a new working directory for your work creating your secure bitcoin wallet contract.

In [ ]:
! mkdir -p secure-bitcoin-wallet

In [ ]:
%cd secure-bitcoin-wallet

---

### Step 7.2 Create a compose subdirectory

Create a subdirectory for your work setting up your contract and change into it

In [ ]:
! mkdir -p {environment,workload/compose}

In [ ]:
%cd workload/compose

---

### Step 7.3 Create your Docker Compose file

Create the Docker Compose file for the secure bitcoin wallet application.

In [ ]:
%%bash
cat << EOF > docker-compose.yml
services:
  wallet:
    image: $(cat ../../../sbw_image.env)
    ports:
      - "443:443"
    environment:
      - WALLET_NAME=general
      - ZHSM=\${ZHSM}
      - APIKEY=\${APIKEY}
EOF

---

### Step 7.4 Check the contents of your Docker compose file

Check on the Docker Compose file you just created with the following code box.

In [ ]:
! cat docker-compose.yml

---

### Step 7.5 Move to the main workload directory

Move back a directory to the main workload directory

In [ ]:
%cd ..

---

### Step 7.6 Create a script to encrypt the workload section of the contract

Create a a convenience script to automate creating the encrypted workload section of the contract.

In [ ]:
%%bash
cat << EOF > flow.workload
# Create the workload section of the contract and add the contents in the workload.yaml file.

WORKLOAD_PLAIN=./workload.yaml.plaintext
WORKLOAD=workload.yaml

echo "  type: workload
  compose:
    archive: \$(tar -czv -C compose . | base64 -w0)
  images:
    dct:
      \$(cat ../../sbw_image.env | cut -d "@" -f1):
        notary: https://notary.docker.io
        publicKey: \$(cat ../../docker.io-${docker_user}-secure-bitcoin-wallet-public.key)" > \${WORKLOAD_PLAIN}

# Download certificate to encrypt contract for Hyper Protect Container Runtime:
HPCR_rev=11
CONTRACT_KEY=./ibm-hyper-protect-container-runtime-1-0-s390x-\${HPCR_rev}-encrypt.crt
curl https://cloud.ibm.com/media/docs/downloads/hyper-protect-container-runtime/ibm-hyper-protect-container-runtime-1-0-s390x-\${HPCR_rev}-encrypt.crt > \${CONTRACT_KEY}


# Use the following command to create a random password:
PASSWORD_WORKLOAD="\$(openssl rand 32 | base64 -w0)"

# Use the following command to encrypt password with the Hyper Protect Container Runtime Contract Encryption Key:
ENCRYPTED_WORKLOAD_PASSWORD="\$(echo -n "\$PASSWORD_WORKLOAD" | base64 -d | openssl rsautl -encrypt -inkey \$CONTRACT_KEY -certin | base64 -w0)"

# Use the following command to encrypt the workload.yaml file with a random password:
ENCRYPTED_WORKLOAD="\$(echo -n "\$PASSWORD_WORKLOAD" | base64 -d | openssl enc -aes-256-cbc -pbkdf2 -pass stdin -in "\$WORKLOAD_PLAIN" | base64 -w0)"

# Use the following command to get the encrypted section of the contract:
WORKLOAD_ENCRYPTED="hyper-protect-basic.\${ENCRYPTED_WORKLOAD_PASSWORD}.\${ENCRYPTED_WORKLOAD}"

echo ""
echo "See `pwd`/workload.yaml.plaintext to see what was encrypted for the workload section of your contract"
echo ""

echo "\$WORKLOAD_ENCRYPTED" > ../\$WORKLOAD
EOF


---

### Step 7.7 Move to the environment directory

Move to the environment directory to begin working on the environment section of the contract.

In [ ]:
%cd ../environment

--- 

### Step 7.8 Set the instructor-provided API key and HSM URL

Set the instructor-provided API Key by pasting it to the right of the = sign in the code cell below and then running the code cell.

In [ ]:
%set_env api_key=

Set the instuctor-provided HSM URL by pasting it to the right of the = sign in the code cell below and then running the code cell.

In [ ]:
%set_env hsm_url=

---

### Step 7.9 Create a script to encrypt the environment section of the contract

Create a convenience script to automate creating the encrypted environment section of the contract.

In [ ]:
%%bash
cat << EOF > flow.env
# Create the env section of the contract and add the contents in the env.yaml file.
ENV_PLAIN="./env.yaml.plaintext"
ENV="env.yaml"

echo "  type: env
  logging:
    logDNA:
      hostname: \${logdna_hostname}
      ingestionKey: \${logdna_ingestion_key}
      port: 6514
  env:
    ZHSM: ${hsm_url}
    APIKEY: ${api_key}" > \${ENV_PLAIN}

cat ./pubSigningKey.yaml >> \${ENV_PLAIN}

# Download certificate to encrypt contract for Hyper Protect Container Runtime:
HPCR_rev=11
CONTRACT_KEY=./ibm-hyper-protect-container-runtime-1-0-s390x-\${HPCR_rev}-encrypt.crt
curl https://cloud.ibm.com/media/docs/downloads/hyper-protect-container-runtime/\$CONTRACT_KEY > \$CONTRACT_KEY


# Use the following command to create a random password:
PASSWORD_ENV="\$(openssl rand 32 | base64 -w0)"

#  Use the following command to encrypt password with the Hyper Protect Container Runtime Contract Encryption Key:
ENCRYPTED_ENV_PASSWORD="\$(echo -n "\$PASSWORD_ENV" | base64 -d | openssl rsautl -encrypt -inkey \$CONTRACT_KEY -certin | base64 -w0 )"

# Use the following command to encrypt env.yaml with a random password:
ENCRYPTED_ENV="\$(echo -n "\$PASSWORD_ENV" | base64 -d | openssl enc -aes-256-cbc -pbkdf2 -pass stdin -in "\$ENV_PLAIN" | base64 -w0)"

# Use the following command to get the encrypted section of the contract:
ENV_ENCRYPTED="hyper-protect-basic.\${ENCRYPTED_ENV_PASSWORD}.\${ENCRYPTED_ENV}"

echo ""
echo "See `pwd`/env.yaml.plaintext to see what was encrypted for the env section of your contract"
echo ""

echo "\$ENV_ENCRYPTED" > ../\$ENV
EOF


---

### Step 7.10 Move back to the main directory

Move back to the secure-bitcoin-wallet main directory

In [ ]:
%cd ..

---

### Step 7.11 Create scripts for signing the contracts

Create a script for generating keys and one for signing the contracts by running the code cells below.

In [ ]:
%%bash
cat << EOF > flow.prepare
# Use the following command to generate key pair to sign the contract 
openssl genrsa -aes128 -passout pass:test1234 -out private.pem 4096
openssl rsa -in private.pem -passin pass:test1234 -pubout -out public.pem

# The following command is an example of how you can get the signing key:
key=\$(awk -vRS="\n" -vORS="\\\\\n" '1' public.pem)
# echo "  signingKey: \"\${key%\\\\n}\"" > environment/pubSigningKey.yaml
printf "%s" "  signingKey: \"\${key%\\\\n}\"" > environment/pubSigningKey.yaml
EOF

In [ ]:
%%bash
cat << EOF > flow.signature
# combine workload and environment
cat workload.yaml env.yaml | tr -d '\n' > contract.yaml

# Sign the combination from workload and env being approved
echo \$( cat contract.yaml | openssl dgst -sha256 -sign private.pem -passin pass:test1234 | openssl enc -base64) | tr -d ' ' > signature.yaml

# Create user data and add signature:
echo "workload: \$(cat workload.yaml)
env: \$(cat env.yaml)
envWorkloadSignature: \$(cat signature.yaml)" > user_data.yaml

echo ""
echo "import `pwd`/user_data.yaml into User Data or copy and paste from below:"
echo ""

cat user_data.yaml
EOF

---

### Step 7.12 Create a script to run the other scripts

Create a main script to launch the various different scripts, automating the entire process of creating the contract.

In [ ]:
%%bash
cat << EOF > makeContract
. ./flow.prepare
cd workload
. ./flow.workload
cd ../environment
. ./flow.env
cd ..
. ./flow.signature
EOF

---

### Step 7.13 Create the contract

Create the contract by running the contract creation automation script.

In [ ]:
! . ./makeContract

---

### Step 7.14 Set an environment variable for your HPVS for VPC Instance

Set an environment variable with your HPVS for VPC instance name

In [ ]:
%set_env sbw_instance=secure-bitcoin-wallet

---

### Step 7.15 Launch your HPVS for VPC Instance

Create your HPVS for VPC Instance with the following command

In [ ]:
! ibmcloud is instance-create $sbw_instance $vpc $zone bz2e-1x4 $subnet --image $(cat ../hpcr_image.env) --user-data @user_data.yaml

---

### Step 7.16 Check the status of your HPVS for VPC Instance

Check the status of your secure bitcoin wallet instance by checking the logs of your deployment in LogDNA. 

If you have for some reason closed your LogDNA instance, navigate to the IBM Cloud UI (signing in if necessary) via [this link](https://cloud.ibm.com/observe/logging) which will take you to your logging instances. Then, click the blue **Open Dashboard** button in the upper right for the logDNA instance you are using for this lab.

If there are issues with your deployment contact an instructor for help unraveling them. 

---

### Step 7.17 Find the nic for your newly created HPVS for VPC instance



In [ ]:
! ibmcloud is instance-network-interfaces $sbw_instance

---

### Step 7.18 Set nic based on above output

Set an environment variable for your nic based on the above output by pasting the above ID into the code block below after the `=` sign and then running the code block.

In [ ]:
%set_env sbw_nic=

---

### Step 7.19 Create a floating IP for the HPVS for VPC instance

Create a floating IP for the HPVS for VPC Instance to access it via an IP

In [ ]:
! ibmcloud is floating-ip-reserve bitcoin-wallet-fip --nic $sbw_nic

---

### Step 7.20 Edit this markdown cell to create a clickable link to get to your sample application

Once your instance is active, note the value of the _Address_ field from the output from _Step 7.19_.  

Double-click anywhere within this paragraph to go into _edit_ mode of this markdown cell.  Then, in the final line of this markdown cell, change `xx.xx.xx.xx` to your public IP address.  The actual URL is on the right side of the line (between parentheses). The text of the link is on the left side of the line in between square brackets.  You may as well change both occurrences of `xx.xx.xx.xx` for clarity, but the one that matters is the link on the right side.   (If you don't see both sides you haven't succesfully got into edit mode—keep trying or ask an instructor for help).  After you have made the change, run this markdown cell by clicking the *Run* icon in the Jupyter Notebook menu above. This will render the markdown and present you with the properly formatted link. Click the link and you will bring up your sample secure bitcoin wallet application.

[Change the xxx.xxx.xxx.xxx to your public IP and then click here](https://xxx.xxx.xxx.xxx/electrum)

---

## Running the Secure Bitcoin Wallet Application in your HPVS for VPC instance

### Overview

This application is a secure [Electrum](https://electrum.org/#home) Bitcoin wallet found at [this GitHub repository](https://github.com/IBM/secure-bitcoin-wallet). You can use it to store the identities necessary to transact on the Bitcoin public blockchain network. While this version of the wallet is just a proof of concept, running this type of wallet on Hyper Protect enables you to protect your digital assets from bad actors such as malicious server administrators.

*This Electrum Bitcoin wallet functions regularly, while inheriting the security benefits of the Hyper Protect Virtual Server for VPC platform. This ensures that a legitimate version of the application is running (Secure Image Build process) and that administrators cannot compromise the application (Hyper Protect Virtual Server for VPC deployment).*

Here is an overview diagram of the _sample secure bitcoin wallet application_ that you built in this lab: 

![Secure Bitcoin Overview](images/sampleapparch.png)

The application, by default, uses software cryptography in order to generate keys, but it can also be configured to use _Hyper Protect Crypto Services_ in order to generate keys more securely, which is what you have done earlier in this lab when you specified the _ZHSM_ and the _APIKEY_ environment variables in _Step 7.2_.


### Step 8.1 Account Registration

Registering a fake account for lab purposes.

1. Choose `Register` since you won't have an existing account on the newly deployed server.

    ![Secure Bitcoin Wallet Welcome](Bitcoin_Wallet_Images/Bitcoin_Wallet_Homepage.png)

2. Enter the registration details you desire.

    ![User Registration](Bitcoin_Wallet_Images/Wallet_register_info.png)


*Do not worry about the information you enter on the registration page (shown below) as it won't map to anything critical to the real world.  You just need your password to log back in if you log out of the application.*

### Step 8.2 Wallet initialization

It's time to set up your Secure Electrum Bitcoin wallet itself!

1. Bring up the wallet screen

    ![Wallet Start](Bitcoin_Wallet_Images/Click_Wallet_Start.png)

2. Enter in a `password` for your wallet and click `Create Wallet`

    ![Create Wallet](Bitcoin_Wallet_Images/Create_Wallet.png)

3. Load your newly created wallet to enable it

    ![Load Wallet](Bitcoin_Wallet_Images/Load_Wallet.png)

4. Reload the page

    ![Initialized Wallet](Bitcoin_Wallet_Images/Reload_new_wallet.png)

5. Your wallet is ready for use in all of its glory

    ![Locked and Loaded Wallet](Bitcoin_Wallet_Images/Loaded_Wallet_Time.png)

### Step 8.3 Receiving Bitcoins

A wallet without currency is blasphemy, so it's time to fill it on up with fake bitcoins.

1. Go to the `Receive` section of the wallet and get the bitcoin wallet address.  

    ![Wallet receiving bitcoin](Bitcoin_Wallet_Images/Receiving_bitcoin.png)

    *This is the address whose alloted bitcoins you lay claim to.*

2. Open [this bitcoin test faucet site](https://testnet-faucet.com/btc-testnet/) [click on the link] and use it to transfer bitcoin value to the `Receiving address` in your wallet **from step 1 above**.

    ![Transfer Bitcoins](Bitcoin_Wallet_Images/Send_bitcoins_to_wallet.png)

    *Instead of transacting on the "real" bitcoin blockchain network, we are using a [testnet](https://medium.com/myetherwallet/understanding-blockchain-changes-testnets-and-mainnets-c2171a8e835f). This means that the network participants agree that the assets on the network have no value. Testnets are used for testing changes to the network, testing applications and introducing new users to the technical aspects of network. There are testnets for many of the public blockchain networks including of course bitcoin and ethereum. Test faucets (like the one we are using) act as distributors of tokens to other users on the testnet.*

3. Switch back to your wallet tab and go to the `History` section and then reload the page.

    ![Check History](Bitcoin_Wallet_Images/Check_History_Receive_Transaction.png)

4. You should now see your wallet address as the recipient of a `pending transaction` from the bitcoin testnet faucet.

    ![First Transaction History](Bitcoin_Wallet_Images/Transaction_History_no_confirmations.png)

5. After clicking for more information you get a detailed transaction overview

    ![Receive Transaction 0 Confirmations](Bitcoin_Wallet_Images/0_confirmations_block_details.png)

    *The number of [confirmations](https://en.bitcoin.it/wiki/Confirmation) a block receives is the number of blocks deep the block that has included the blockchain transaction is. In other words, `1 confirmation` means that the current head block includes the transaction. `2 confirmations` means that the current head block's parent block contains the transaction and so on. The deeper the transaction is (more confirmations it has) the more mining power it would take to "reverse history" and undo the transaction.*

    Since there are `0 confirmations` in the picture above, it means that the transaction is not yet included in a block. This is why it has no timestamp yet and its block number is listed as `0`.

6. After waiting anywhere from a minute to a few minutes (depending on the current testnet traffic) you should see a confirmation on your transaction.

    *You will need to keep reloading the page for it to update. It doesn't update automatically.*

    ![1 Confirmation Received](Bitcoin_Wallet_Images/1_Confirmation_Received.png)

    There is now a timestamp and a block number!

    *As time passes more confirmations will accumulate. You can check on the `confirmations` again at the end of the lab to show yourself this (Given enough time has passed for one or more additional confirmations to occur).*

### Step 8.4 Returning Bitcoins

Since you are feeling charitable you decide to transfer your testnet value back to the faucet's address.

1. Go back to [the faucet site](https://testnet-faucet.com/btc-testnet/) and get its bitcoin address

    ![Faucet Address](Bitcoin_Wallet_Images/testnet_faucet_address.png)

2. Send bitcoins to faucet using `Send` section of wallet

    ![Send Bitcoins to Faucet](Bitcoin_Wallet_Images/Send_bitcoins_back_to_faucet.png)

3. Once sent you will see the `Signed transaction` information filled out on your wallet page.

    ![Bitcoins Sent Back to Faucet](Bitcoin_Wallet_Images/bitcoin_send_transaction_output.png)

4. You can go back to the [bitcoin faucet site](https://bitcoinfaucet.uo1.net/send.php) (you may have to refresh the page) and see the transaction as green there with a sender of your address and a transaction ID matching the one you sent.

    ![Bitcoins returned to faucet](Bitcoin_Wallet_Images/transaction_giving_coins_back_to_faucet.png)

5. You can view both the initial received transaction and the newly sent transaction in the `History` tab of your bitcoin wallet.

    *You have to keep reloading the page for it to update with new confirmations. If you wait long enough, eventually these transactions will gain confirmations as in the picture below*

    ![Both transactions with 1 confirmation](Bitcoin_Wallet_Images/both_confirmed_transactions.png)

6. You can continue to play around with the electrum wallet before moving on.

### Summary

You have participated in the bitcoin testnet using a securely built electrum bitcoin wallet powered by a Hyper Protect Virtual Server. Congratulations!!! 

Now, it's time to clean up..

---

## Clean up when you are done with the lab

This section describes important cleanup steps to perform when you have completed the lab. It will guide you through deleting the following artifacts that you created during the lab:

1. The Secure Build Server HPVS for VPC instance you created
2. The Secure Build Server volume you created
3. The Sample Secure Bitcoin Wallet Application HPVS for VPC instance you created
4. The subnet you created for this lab
5. The public gateway you created for this lab
6. The 2 floating ips you creataed for this lab
7. The VPC you created for this lab
8. The logDNA instance you created for this lab (skip if using existing instance)
9. The Secure Bitcoin Wallet Application container image that you created for this lab
10. The Docker Access Token that you created for this lab
11. The GitHub ssh key you created for this lab


---

### Step 9.1 Delete your Secure Build Server HPVS for VPC instance

Run the following code block to delete your secure build Server HPVS for VPC instance.

In [ ]:
! ibmcloud is instance-delete $instance --force

---

### Step 9.2 Delete your Secure Build Server volume

Run the following code block to delete your secure build server volume.

In [ ]:
! ibmcloud is volume-delete secure-build-volume --force

---

### Step 9.3 Delete your sample Secure Bitcoin Wallet Application HPVS for VPC instance

Running the following code cell to delete your secure bitcoin for application.

In [ ]:
! ibmcloud is instance-delete $sbw_instance --force

### Step 9.4 Delete the subnet for your VPC

Run the following code block to delete the subnet you created for this lab. If you get a failed to delete you may need to wait a minute or so for the instances to be fully deleted before re-running this command.

In [ ]:
! ibmcloud is subnet-delete $subnet --force

---

### Step 9.5 Delete your public gateway

Run the following code block to delete the public gateway you created for this lab.

In [ ]:
! ibmcloud is public-gateway-delete $gateway --force

---

### Step 9.6 Deleting the Floating IPS you created for this lab

Run the following two code blocks to delete the floating IPs you created for this lab.

In [ ]:
! ibmcloud is floating-ip-release secure-build-fip --force

In [ ]:
! ibmcloud is floating-ip-release bitcoin-wallet-fip --force

---

### Step 9.7 Delete the VPC you created for this lab

Run the following code block to delete the VPC you created for this lab.

In [ ]:
! ibmcloud is vpc-delete $vpc --force

---

### Step 9.8 Delete the logDNA instance you created for this lab

Run the following code block to delete the logDNA instance you created for this lab.

In [ ]:
! ibmcloud resource service-instance-delete secure-build-logdna --force

---

### Step 9.9 Delete your sample secure bitcoin wallet application secure image from Docker Hub

While logged in to your Docker Hub account, follow these steps from the Docker Hub web UI in order to delete the image you created in this lab:

1. Click the *Repositories* link in the banner at the top of the page.
2. Click the image that you created in the lab.
3. Click the _Settings_ link.
4. Scroll down to the _Delete Repository_ section and click the _Delete repository_ button.
5. Enter your image name as confirmation–copy and paste is your friend here– for confirmation and click the _Delete_ button.



---

### Step 9.10 Delete the Docker Access Token you created for this lab

Follow [these instructions](https://docs.docker.com/docker-hub/access-tokens/#modify-existing-tokens) to delete the Docker Access Token you created earlier in the lab.



---

### Step 9.11 Remove the ssh key you created for this lab from your GitHub account

Delete the ssh key you added for the lab from your GitHub account by following steps 1-3 of [these instructions](https://docs.github.com/en/github/authenticating-to-github/reviewing-your-ssh-keys), choosing the _Delete_ button for the ssh key you want to get rid of.  If you were an existing GitHub user prior to this lab, be careful not to delete any other ssh keys you may have added for other purposes.

---

## You have reached the end of the lab!

Thank you for taking this lab, and please feel free to provide feedback, either by opening a GitHub issue on the lab repo at https://github.com/ibm-wsc/ibm-cloud-hpvs-sbs-byoi-lab or by sending an email directly to either, Barry Silliman, at "silliman at us dot ibm dot com" or Garrett Woodworth at "garrett dot lee dot woodworth at ibm dot com  (the email spammers will never figure out our real email addresses from that, but you will).

---

## Acknowledgements

Our esteemed colleague Jin VanStee also made significant contributions to this lab.

## References

This lab is based largely on publicly available material on IBM's website, especially [Tutorial: Using Secure Build Server with a digital wallet](https://cloud.ibm.com/docs/hp-virtual-servers?topic=hp-virtual-servers-tutorial_secure_build_server)

Additional information about using the Secure Build Server CLI can be found [here](https://github.com/ibm-hyper-protect/secure-build-cli)  

Here is the [GitHub repo for the sample secure bitcoin wallet application](https://github.com/IBM/secure-bitcoin-wallet)